# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [26]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=0"
#URL = "http://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=jobsite&salary=%2420%2C000%2B&radius=25&l=New+York&fromage=any&limit=50&sort=&psf=advsrch"

In [27]:
import requests
import bs4
import urllib2
from bs4 import BeautifulSoup

In [41]:
## YOUR CODE HERE

title=[]
company=[]
location=[]
salary=[]
summary=[]

def scraper(URL):
    dom = BeautifulSoup(requests.get(URL).text,"lxml")

    jobDivs = dom.find_all("div", {"class":"result","class":"row"})

    for div in jobDivs:
        divSoup = BeautifulSoup(unicode.join(u'\n',map(unicode,div)),"lxml")

        #get title
        title.append(divSoup.find("a",{"data-tn-element":"jobTitle"})['title'])

        #get company
        company1 = divSoup.find("a",{"data-tn-element":"companyName"})
        company2 = divSoup.find("span",{"class":"company"}).string
        company3 = divSoup.find("span",{"itemprop":"name"})
        if(company1 != None):
            #print "1: ",company1.contents[-1].strip()
            company.append(company1.contents[-1].strip())
        elif(company2 != None):
            #print "2: ",company2.strip()
            company.append(company2.strip())
        elif(company3 != None):
            if(company3.string != None):
                #print "3a: ",company3.string.strip()
                company.append(company3.string.strip())
            else:
                #print "3b: ",company3.contents[-1].string.strip()
                if(company3.contents[-1].string != None):
                    company.append(company3.contents[-1].string.strip())
                else:
                    company.append("What:" + company3.contents[-1].text)
                    
        else:
            print "N/A"

        #get location 
        locationString = divSoup.find("span",{"class":"location"}).string
        if (locationString == None):
            locationString = divSoup.find("span",{"itemprop":"addressLocality"}).contents[0].strip()
        location.append(locationString)

        #get salary

        salaryItem = divSoup.find("span",{"class":"location"}).findNextSibling("div")
        if (salaryItem != None):
            salary.append(salaryItem.text)
        else:
            salary.append("N/A")

        #get summary

        #print divSoup.find("span",{"class":"summary"}).text
        #summary.append(divSoup.find("span",{"class":"summary"}).text)



Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [42]:
max_results_per_city = 100

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', "Atlanta"]):
    for start in range(0, max_results_per_city, 10):
        url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l="+ city +"&start=" + str(start)
        scraper(url_template)

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [43]:
import pandas as pd

In [47]:
pd.DataFrame({'title':title, 'company':company, 'location':location, 'salary':salary})

,company,location,salary,title
0,Workday,"San Francisco, CA",N/A,Data Scientist
1,Apple,"San Francisco, CA",N/A,"Software Engineer, iTunes- Big Data Play Activity"
2,"Acumen, LLC","Burlingame, CA",N/A,Data and Policy Analyst - Statistical Programmer
3,Zynga,"San Francisco, CA",N/A,Data Scientist
4,Ayasdi,"Menlo Park, CA 94025",N/A,Data Scientist (Healthcare)
5,Cloudmellow,"San Francisco, CA",N/A,"Data Scientists,San Francisco, CA"
6,Apple,"San Francisco, CA 94114",N/A,"Data Scientist, Siri Search Team"
7,OnboardIQ,"San Francisco, CA",N/A,Algorithm and Data Scientist (Machine Learning)
8,Komodo Health,"San Francisco, CA",N/A,Data Scientist
9,ZestFinance,"San Francisco, CA",N/A,Sr. Machine Learning Modeler / Data Scientist


Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [14]:
## YOUR CODE HERE

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [12]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [15]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [16]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [17]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [18]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [20]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [21]:
## YOUR CODE HERE

In [22]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE